# Introduction to ForneyLab

ForneyLab is a toolbox for deriving solutions to inference problems by message passing on Forney-style factor graphs. A Forney-style factor graph (FFG) represents a generative model, and message passing solutions to inference questions can be automatically derived from this generative model definiton.

We designed ForneyLab to be practical, while retaining maximal flexibility. The inherent modularity of the FFG framework makes ForneyLab extensible at all levels (nodes, update rules, algorithms, inference engines). In the end, ForneyLab generates a block of code that infers the solution to the inference question. This resulting program can ultimately be optimized by the user.

The ForneyLab approach to solving inference problems consists of three phases:

1. Build (the generative model)
2. Schedule (the message passing algorithm)
3. Infer (the marginal distributions)

Each of the demos (with exception of the composite node demo) steps through these pases in turn, showcasing the most important aspects of ForneyLab functionality. For more detailed information we refer to the Julia help functionality (simply type `?` and the ForneyLab function you're interested in), or the source code itself.

# Build

The central concept in ForneyLab is the `Variable` type, and the build phase specifies the relations between `Variable`s. Through the generative model definition, ForneyLab builds the corresponding Forney-style factor graph (FFG). In an FFG,  edges represent variables, and nodes represent the factor functions that encode the relations between variables.

After including ForneyLab and indicating the start of a new `FactorGraph` definition, we can build a simple model through the `@RV` syntax.

In [1]:
using ForneyLab

# Start a new graph
g = FactorGraph()

# Build the model
@RV x ~ GaussianMeanVariance(0.0, 1.0)

ForneyLab.Variable(:x, Edges:
Edge belonging to variable x: ( gaussian_1.i[out] )----( NONE ).
)

We can inspect the corresponding FFG with the `draw` function. We observe a Gaussian distributed variable `x` with the mean and variance clamped to fixed values by two `Clamp` nodes. 

In [2]:
# Inspect the graph
ForneyLab.draw(g)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.38.0 (20140413.2041)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 8430305611620705538 -->
 8430305611620705538 
 
 𝒩 
 gaussian_1 
 
<!-- 6118021666468662668 -->
 6118021666468662668 
 
 clamp_1 
 
<!-- 8430305611620705538--6118021666468662668 -->
 8430305611620705538--6118021666468662668 
 
 clamp_1 
 1 out 
 2 m 
 
<!-- 14840921163602855047 -->
 14840921163602855047 
 
 clamp_2 
 
<!-- 8430305611620705538--14840921163602855047 -->
 8430305611620705538--14840921163602855047 
 
 clamp_2 
 1 out 
 3 v 
 
<!-- 63613161493043712542 -->
 63613161493043712542 
 
<!-- 63613161493043712542--8430305611620705538 -->
 63613161493043712542--8430305611620705538 
 
 x 
 1 out

# Schedule

Suppose that we are interested in the marginal distribution over `x`. This (trivial) inference question can be addressed by scheduling a sum-product algorithm towards `x`. We see that the schedule requires the computation of three messages. This schedule can be visualized in the graph for the generative model, where for brevety the messages coming from `Clamp` nodes are omitted.

In [3]:
# Schedule sum-product inference
schedule = sumProductSchedule(x)
println(schedule)

	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_1
	SPClamp{Univariate} on Interface 1 (out) of ForneyLab.Clamp{ForneyLab.Univariate} clamp_2
1.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of ForneyLab.GaussianMeanVariance gaussian_1



In [4]:
ForneyLab.draw(g, schedule=schedule)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.38.0 (20140413.2041)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 8430305611620705538 -->
 8430305611620705538 
 
 𝒩 
 gaussian_1 
 
<!-- 6118021666468662668 -->
 6118021666468662668 
 
 clamp_1 
 
<!-- 8430305611620705538--6118021666468662668 -->
 8430305611620705538--6118021666468662668 
 
 clamp_1 
 1 out 
 2 m 
 
<!-- 14840921163602855047 -->
 14840921163602855047 
 
 clamp_2 
 
<!-- 8430305611620705538--14840921163602855047 -->
 8430305611620705538--14840921163602855047 
 
 clamp_2 
 1 out 
 3 v 
 
<!-- 63613161493043712542 -->
 63613161493043712542 
 
<!-- 63613161493043712542--8430305611620705538 -->
 63613161493043712542--8430305611620705538 
 
 x 
 1 out 
 (1)

This visualization is convenient for debugging algorithms, but we can also immediately convert our schedule to Julia-executable code.

In [5]:
algo = sumProductAlgorithm(x)
println(algo)

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(1))

messages[1] = ruleSPGaussianMeanVarianceOutVPP(nothing, Message(Univariate, PointMass, m=0.0), Message(Univariate, PointMass, m=1.0))

marginals[:x] = messages[1].dist

return marginals

end


# Infer

In the infer step we execute our automatically generated code. We simply evaluate it and inspect the result, which is the marginal `ProbabilityDistribution` over `x`. Although this is a trivial result, it showcases the build-schedule-infer paradigm in an instinctive manner. The following demos will explore more complex models and inference algorithms.

In [6]:
eval(parse(algo))
data = Dict() # No data are available
marginals = step!(data)
marginals[:x]

𝒩(m=0.00, v=1.00)
